# Select to Config

## 패키지 및 데이터 로딩

### 패키지 로딩

In [7]:
# data handling
import numpy as np
import pandas as pd

# Imputation
from sklearn.ensemble import RandomForestRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# scaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import RobustScaler

# main algorithms
from catboost import CatBoostClassifier

# making event
from itertools import product

# scoring
import sklearn.metrics as metrics
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

# visualization processing
from tqdm import notebook
import time

# ignore warnings
import warnings
warnings.filterwarnings("ignore")

### 데이터 로딩

In [8]:
data = pd.read_csv('C:/bad_web/data/train_dataset.csv')
data.drop(columns = ['Unnamed: 0'], inplace = True)
data['Result_v1'] = data['Result_v1'].apply(lambda x: -1 if x == 'malicious' else 1)

## Config

### 우리에게 필요한 환경에는 어느 것이 있을까?

In [9]:
# 경우의 수 분석하기
one_column_select = [True,False] # 하나의 값만 가지는 칼럼을 분석에 적용할 것인지 판단
url_len_select = [True,False] # url 길이를 분석에 적용할 것인지 판단
url_entropy_select = [True,False] # url entropy를 분석에 적용할 것인지 판단
url_port_select = [True,False] # url port를 분석에 적용할 것인지 판단
iframe_select = [True,False] # iframe을 분석에 적용할 것인지 판단
head_select = [True,False] # head를 분석에 적용할 것인지 판단
body_select = [True,False] # body를 분석에 적용할 것인지 판단
one_distribute_select = [True,False] # error에만 분포를 가지는 칼럼의 분포를 분석에 적용할 것인지 판단
drop_duplicate_select = [True,False] # 중복된 데이터를 제거할 것인지 판단
missing_impute_select = [True,False] # missing value를 impute할 것인지 판단
scaler_select = ["NONE"] # scaler를 적용할 것인지 판단

# 모든 경우의 수를 product 연산을 통해 생성하기
iter = list(product(one_column_select,url_len_select,url_entropy_select,url_port_select,
iframe_select,head_select,body_select,one_distribute_select,
drop_duplicate_select,missing_impute_select,scaler_select))

### 데이터 셋 모델링 함수 구축

In [10]:
# 데이터 셋 모델링 함수 구축
def select_dataset(data,config,test_size) :
    """
    config : dict
    test_size : float : test_size를 결정
    """
    # 데이터의 인덱스 초기화
    data.reset_index(drop = True, inplace = True)

    # one_columns에 해당하는 칼럼 정의
    one_columns = ['url_chinese_present',"html_num_tags('applet')"]
    # one_distribute에 해당하는 칼럼 정의
    one_distribute = ['url_query_len','url_num_query_para']
    # 모든 칼럼에서 경우의 수에 따라서 필요 없는 칼럼을 제거해 나갈 것이다.
    all_columns = list(data.columns)
    # one_columns에 해당하는 칼럼을 제거할 것인가?
    if config['one_column_on'] == False :
        for i in one_columns :
            all_columns.remove(i)
    # url_len을 분석에 적용할 것인가?    
    if config['url_len_off'] == False :
        all_columns.remove('url_len')
    # url_entropy를 분석에 적용할 것인가?
    if config['url_entropy_off'] == False :
        all_columns.remove('url_entropy')
    # url_port를 분석에 적용할 것인가?
    if config['url_port_off'] == False :
        all_columns.remove('url_port')
    # iframe을 분석에 적용할 것인가?
    if config['iframe_off'] == False :
        all_columns.remove("html_num_tags('iframe')")
    # head를 분석에 적용할 것인가?
    if config['head_off'] == False :
        all_columns.remove("html_num_tags('head')")
    # body를 분석에 적용할 것인가?
    if config['body_off'] == False :
        all_columns.remove("html_num_tags('body')")
    # one_distribute에 해당하는 칼럼을 분석에 적용할 것인가?
    if config['one_distribute_on'] == False :
        for i in one_distribute :
            all_columns.remove(i)

    # 경우의 수에 따라 정제된 칼럼을 정의함
    data = data[all_columns]

    # 중복된 데이터를 제거할 것인가?
    if config['drop_duplicate'] == True :
        data.drop_duplicates(inplace = True)
        data.reset_index(drop = True, inplace = True)
    
    # missing value를 impute할 것인가?
    if config['missing_imput'] == True :
        X = data.drop(columns = ["Result_v1"])
        y = data["Result_v1"]
        imp = IterativeImputer(estimator = RandomForestRegressor(verbose=0, random_state = 42), # impute를 할 시 InterativeImputer를 사용(MissForest)
                               max_iter=10,
                               verbose = 0,
                               imputation_order= 'ascending',
                               random_state=0)
        X_imp=pd.DataFrame(imp.fit_transform(X))
        X_imp.columns = X.columns
        data = pd.concat([X_imp,y], axis = 1)    
    else :
        data = data.dropna() # impute 하지 않을 시, 모든 missing value를 제거
        data = data.reset_index(drop = True)

    # train test split
    train, test = train_test_split(data, test_size = test_size, random_state = 42,shuffle=True)
    train.reset_index(drop = True, inplace = True)
    test.reset_index(drop = True, inplace = True)
    
    # 어느 scaler를 적용할 것인가?
    if config['scaler'] == "MinMax" :
        scaler = MinMaxScaler()
        train_x = train.drop(columns = ["Result_v1"])
        column = train_x.columns
        train_y = train["Result_v1"]
        test_x = test.drop(columns = ["Result_v1"])
        test_y = test["Result_v1"]
        scaler = scaler.fit(train_x)
        train_x = scaler.transform(train_x)
        test_x = scaler.transform(test_x)
        train_x = pd.DataFrame(train_x)
        test_x = pd.DataFrame(test_x)
        train_x.columns = column
        test_x.columns = column
        train = pd.concat([train_x,train_y],axis = 1)
        test = pd.concat([test_x,test_y],axis = 1)
    if config['scaler'] == "Standard" :
        scaler = StandardScaler()
        train_x = train.drop(columns = ["Result_v1"])
        column = train_x.columns
        train_y = train["Result_v1"]
        test_x = test.drop(columns = ["Result_v1"])
        test_y = test["Result_v1"]
        scaler = scaler.fit(train_x)
        train_x = scaler.transform(train_x)
        test_x = scaler.transform(test_x)
        train_x = pd.DataFrame(train_x)
        test_x = pd.DataFrame(test_x)
        train_x.columns = column
        test_x.columns = column
        train = pd.concat([train_x,train_y],axis = 1)
        test = pd.concat([test_x,test_y],axis = 1)
    if config['scaler'] == "MaxAbs" :
        scaler = MaxAbsScaler()
        train_x = train.drop(columns = ["Result_v1"])
        column = train_x.columns
        train_y = train["Result_v1"]
        test_x = test.drop(columns = ["Result_v1"])
        test_y = test["Result_v1"]
        scaler = scaler.fit(train_x)
        train_x = scaler.transform(train_x)
        test_x = scaler.transform(test_x)
        train_x = pd.DataFrame(train_x)
        test_x = pd.DataFrame(test_x)
        train_x.columns = column
        test_x.columns = column
        train = pd.concat([train_x,train_y],axis = 1)
        test = pd.concat([test_x,test_y],axis = 1)
    if config['scaler'] == "Robust" :
        scaler = RobustScaler()
        train_x = train.drop(columns = ["Result_v1"])
        column = train_x.columns
        train_y = train["Result_v1"]
        test_x = test.drop(columns = ["Result_v1"])
        test_y = test["Result_v1"]
        scaler = scaler.fit(train_x)
        train_x = scaler.transform(train_x)
        test_x = scaler.transform(test_x)
        train_x = pd.DataFrame(train_x)
        test_x = pd.DataFrame(test_x)
        train_x.columns = column
        test_x.columns = column
    if config['scaler'] == "NONE" :
        train_x = train.drop(columns = ["Result_v1"])
        train_y = train["Result_v1"]
        test_x = test.drop(columns = ["Result_v1"])
        test_y = test["Result_v1"]
    return train_x, train_y, test_x, test_y # 최종적으로 train_x, train_y, test_x, test_y를 return

### 평가함수 구축

In [11]:
def iter_config(one_column,url_len,url_entropy,url_port,iframe,head,body,one_distribute,drop_duplicate,missing_imput,scaler) :
    # config 딕셔너리 만들기
    config = dict()
    config['one_column_on'] = one_column
    config['url_len_off'] = url_len
    config['url_entropy_off'] = url_entropy
    config['url_port_off'] = url_port
    config['iframe_off'] = iframe
    config['head_off'] = head
    config['body_off'] = body
    config['one_distribute_on'] = one_distribute
    config['drop_duplicate'] = drop_duplicate
    config['missing_imput'] = missing_imput
    config['scaler'] = scaler
    # 환경에 따른 데이터 셋 불러오기
    train_x, train_y, test_x, test_y  = select_dataset(data,config,0.2)
    # catboost 모델 학습 및 예측
    model = CatBoostClassifier(random_state = 42, verbose = 0)
    model.fit(train_x, train_y)
    predict_y = model.predict(test_x)
    # f1 score로 평가
    f1 = metrics.f1_score(test_y, predict_y)
    return f1

### 결과 추출

In [ ]:
result = []
for i in notebook.tqdm(iter) :
    time.sleep(0.01)
    output = iter_config(i[0],i[1],i[2],i[3],i[4],i[5],i[6],i[7],i[8],i[9],i[10])
    result.append(output)
    print(i,output)

### 데이터프레임에 결과 저장

In [ ]:
# 데이터 프레임 생성 및 값 넣기
final = pd.DataFrame()
final["iter"] = iter
final["f1"] = result
# 스코어가 높은 순서대로 sorting하기 & csv 파일 형태로 저장
final.sort_values(by = "f1", ascending = False).to_csv("result_all.csv")